In [59]:
from urllib.error import HTTPError
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import codecs
import tqdm
import pickle

In [60]:
# 게시물 1개 수집하는 함수
def brunch_to_html(url):
    # 웹페이지 수집
#     soup = BeautifulSoup(urlopen(url), 'html.parser')
    try:
        soup = BeautifulSoup(urlopen(url), 'html.parser')
    except HTTPError:
        return None
    except Exception as e:
        print(type(e))
        
    articles = soup.findAll(re.compile(r'(p|div|h[1-6]|blockquote)'),     {'class':re.compile(r'(wrap_item item_type_text|wrap_item item_type_bullet|wrap_img_float)')})
#     articles[9].text
    ## 제목 추출
    title = soup.find('h1', {'class':'cover_title'}).text
    ## 소제목 추출
    sub_title = soup.find('p', {'class':'cover_sub_title'}).text
    # 결과물의 body 생성
    body = '<h1>' + title + '</h1><br><h3>' + sub_title + '</h3><br>'
    # 본문에서 텍스트와 그림 링크를 추출하여 body 생성
    for line in articles:
        if (line.text != '\xa0'):# & (line.text != ''):
            if str(line).__contains__('div'):
                try:
                    img_link = re.search(r'http.*', line.find('img')['src']).group(0)
                except:
                    img_link = ''
                try:
                    caption = line.find('span', {'class':'text_caption'}).text
                except:
                    caption = ''
                body += '<center><img src="' + img_link + '"><br>' + caption + '</center><br>'
            elif str(line).__contains__('h2'):
                body += '<br><h2>' + line.text + '</h2><br>'
            elif str(line).__contains__('h3'):
                body += '<h3>' + line.text + '</h3>'
            elif str(line).__contains__('h4'):
                body += '<h4>' + line.text + '</h4>'
            elif str(line).__contains__('h5'):
                body += '<h5>' + line.text + '</h5>'
            elif str(line).__contains__('h6'):
                body += '<h6>' + line.text + '</h6>'    
            elif str(line).__contains__('blockquote'):
                if str(line).__contains__('type1'):
                    body += '<br><center><h3><font color="grey">' + line.text + '</font></h3></center><br>'
                elif str(line).__contains__('type2'):
                    line_text = re.sub(r"(<blockquote[^>]*>|</blockquote>)", '', str(line)).                    replace('</br>', '<br>')
                    body += '<br><div style="border-color:#acacac;border-width:0 0 0 2px;border-style:solid;padding:1px 0 0 12px;color:#666;line-height:18pt;text-align:left}"><font color="grey">' + line_text + '</font></div><br>'
                elif str(line).__contains__('type3'):
                    line_text = re.sub(r"(<blockquote[^>]*>|</blockquote>)", '', str(line)).                    replace('</br>', '<br>')
                    body += '<br><div style="border:1px solid #d7d7d7;text-align:left;padding:21px 25px 20px;color:#666;line-height:18pt}"><font color="grey">' + line_text + '</font></div><br>'
                else:
                    body += '<br><font color="grey">' + line.text + '</font><br>'
            elif str(line).__contains__('bullet'):
                body += '<li>' + line.text + '</li>'
            else:
                line_text = re.sub(r'(<blockquote[^>]*>|</blockquote>|<p[^>]*>|</p>)', '', str(line)).                    replace('<br/>', '<br>').replace('<b></b>', '')
                if line_text != '<br>':
                    #print(line_text)
                    body += '<br>' + line_text + '<br>'
            
    # 전체 html 파일 생성
    pad_front = '<html><head><style>' + '</style></head><body>'
    pad_rear = '</body></html>'
    html = pad_front + body + pad_rear

    return(html)

### urls in excel

In [25]:
excel_path = './brunch_URL.xlsx'
raw = pd.read_excel(excel_path, engine='openpyxl', 
                    sheet_name=None ## all sheets
                   )

In [26]:
url_dict = {}

for key in raw.keys():
    df = raw[key]
    df.dropna(inplace=True) ### nan 값 제거
    df = pd.DataFrame(df.loc[:, 'URL'].str.split('@', 1).tolist(), columns = ['title', 'id&order'])
    id_and_order = pd.DataFrame(df['id&order'].str.split('/', 1).tolist(), columns = ['id', 'order'])
    url_dict[key] = id_and_order

### import html files using the urls

In [27]:
import kss

In [28]:
def remove_tag(content):
   cleanr =re.compile('<.*?>')
   cleantext = re.sub(cleanr, '', content)
   return cleantext

* 개별 pickle 파일

In [46]:
base_url = 'https://brunch.co.kr/'
# section_name = '휴가'

for key in tqdm.tqdm(url_dict.keys()):
    section_name = key
    section_dir = os.path.join(os.getcwd(), 'scraped', section_name)
    if not os.path.exists(section_dir):
        os.makedirs(section_dir)
        
    id_and_order = url_dict[section_name]
    
    for idx, row in tqdm.tqdm(id_and_order.iterrows()):
        user_id = row['id']
        page_order = row['order']
        url = base_url + '@' + user_id + '/' + page_order

        tmp_str = brunch_to_html(url)
        
        if tmp_str is None:
            continue
        
        tmp_str = tmp_str.replace("\xa0", "")
        tmp_str = tmp_str.replace("\udf7d", "")
        tmp_str = tmp_str.replace("\udc51", "")
#         tmp_str = tmp_str.encode('unicode_escape').decode('ascii')
#         tmp_str = re.sub('\\\\\S\S\S\S', '', tmp_str)
        
        tmp_str = remove_tag(tmp_str)
        
        tmp_str = kss.split_sentences(tmp_str)

#         save_path = os.path.join(section_dir, user_id + '-' + page_order + '.txt')
        save_path = os.path.join(section_dir, user_id + '-' + page_order + '.pickle')
        with open(save_path, 'wb') as f:
            pickle.dump(tmp_str, f)

        
#         with open(save_path, 'w') as f:
#             f.write(tmp_str)
            
#         break
        
#     break

  0%|          | 0/7 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  5.41it/s]
2it [00:00,  5.60it/s]
3it [00:00,  6.10it/s]
4it [00:00,  4.48it/s]
5it [00:01,  4.09it/s]
6it [00:01,  4.52it/s]
8it [00:01,  6.02it/s]
9it [00:01,  5.48it/s]
10it [00:02,  4.79it/s]
11it [00:02,  4.58it/s]
12it [00:02,  4.84it/s]
13it [00:02,  4.51it/s]
14it [00:02,  4.58it/s]
15it [00:03,  4.33it/s]
16it [00:03,  4.66it/s]
17it [00:03,  4.30it/s]
18it [00:03,  4.10it/s]
19it [00:04,  4.24it/s]
20it [00:04,  4.53it/s]
22it [00:04,  4.79it/s]
23it [00:04,  5.27it/s]
25it [00:05,  4.78it/s]
26it [00:05,  4.15it/s]
27it [00:05,  3.95it/s]
28it [00:06,  3.92it/s]
29it [00:06,  3.94it/s]
30it [00:06,  3.71it/s]
31it [00:07,  3.64it/s]
32it [00:07,  4.28it/s]
33it [00:07,  4.72it/s]
34it [00:07,  5.01it/s]
35it [00:07,  4.82it/s]
37it [00:08,  5.45it/s]
38it [00:08,  6.07it/s]
39it [00:08,  5.67it/s]
40it [00:08,  4.16it/s]
41it [00:08,  4.25it/s]
42it [00:09,  4.50it/s]
43it [00:09,  4.49it/s]
44it [00:09,  4

KeyboardInterrupt: 

In [101]:
url

'https://brunch.co.kr/@ccllawyer/9'

In [119]:
with open('./scraped/연봉/buildingking-62.pickle', 'rb') as f:
    a = pickle.load(f)
a

['나는 얼마짜리 사람인가?',
 '예전에 네이버의 이해진 의장님이 이런 말씀을 하신 적이 있습니다.',
 '출처: 바이라인 네트워크https://byline.network/2016/07/1-252/"여러분들 네이버란 간판 떼고 나가서 아무서비스나 한번 만들어보세요.몇 명이나 써줄 것 같아요? 10명이나 다운 받아줄까요?지금 네이버란 간판이 여러분들이 만드는 서비스를 지켜주고 있다는 걸 깨닫게 될 거예요.못 믿겠으면 나가서 한 번 해보세요. 그게 여러분들 진짜 실력이에요."가슴이 두근두근 뛰고 약간의반항심도 들었던 것 같습니다.',
 '저 말이 진짜일까?',
 '아니, 난 잘할 수 있을 것 같은데.',
 '이 이야기는 제 가슴에 꽂혀 오랜 시간이 지나도록 잊히지 않았습니다.',
 '회사 다닐 때 이런 생각을 가끔씩 했던 것 같습니다.',
 '내가 회사 다니면서 연봉 1억을 받는다면 사실 내가 회사에 가져다준 가치는 그 이상이야. 적어도 1억 5천은 되겠지.',
 '회사는 내게서 5천만 원을 수수료처럼 떼 가고 있는 거야. 그렇다면 내가 회사를 그만두고 혼자 일한다면 1억 5천 이상을벌 수 있는 사내가 되겠군.그리고 이런 생각은 이해진 의장님의 이야기와 항상 겹쳐지면서 저는 정말 궁금했습니다.',
 '내가 나만의 서비스를 만든다면 결과가 어떨까?',
 '내 실력은 진짜일까 가짜일까.',
 '남이 만든 회사가 아니라 내가 창조해낸 제품으로 단 돈 만원이라도 벌어보고 싶다.',
 '딱 만원만 벌고 실패해도 좋으니 내 진짜 실력이라는 걸테스트해보고 싶어.',
 '어릴 때 스포츠 선수들이 연봉 때문에 재계약을 안 하고 싸우는 것을 보면 이상했습니다.',
 '돈을 저렇게 많이 받는데 그깟 몇 프로 더 받으려고 저렇게 싸워가면서 뻐팅긴다고?이런 질문을 어른들에게 하면어김없이 이런 대답이 돌아왔습니다.',
 '"스포츠 선수들은 연봉을 실력으로 생각하는 거야. 자기실력을 인정받고 싶은 거지."이제는 조금 이해할 수 있을 것 같습니다.',
 '이 세상에서 진실을 테스트할 수 있

* 1개 pickle 파일로 통합

In [50]:
from collections import defaultdict

In [51]:
txt_dict = {}

In [ ]:
base_url = 'https://brunch.co.kr/'
# section_name = '휴가'

for key in tqdm.tqdm(url_dict.keys()):
    section_name = key
#     section_dir = os.path.join(os.getcwd(), 'scraped', section_name)
#     if not os.path.exists(section_dir):
#         os.makedirs(section_dir)
        
    id_and_order = url_dict[section_name]
    txt_dict[section_name] = defaultdict(dict)
    
    for idx, row in id_and_order.iterrows():
        user_id = row['id']
        page_order = row['order']
        url = base_url + '@' + user_id + '/' + page_order

        tmp_str = brunch_to_html(url)
#         try:
#             tmp_str = brunch_to_html(url)
#         except HTTPError as e:
#             continue
        
        if tmp_str is None:
            continue
            
        tmp_str = remove_tag(tmp_str)
        
        tmp_str = tmp_str.replace("\xa0", "")
        tmp_str = tmp_str.replace("\udf7d", "")
        tmp_str = tmp_str.replace("\udc51", "")
#         tmp_str = re.sub('\\\\ud\S\S', '', tmp_str)
        
        tmp_str = kss.split_sentences(tmp_str)
        
#         save_path = os.path.join(section_dir, user_id + '-' + page_order + '.txt')
#         save_path = os.path.join(section_dir, user_id + '-' + page_order + '.pickle')
#         with open(save_path, 'wb') as f:
#             pickle.dump(tmp_str, f)

        
#         with open(save_path, 'w') as f:
#             f.write(tmp_str)
        
        txt_dict[section_name][user_id + '-' + page_order] = tmp_str
    
#         break
        
#     break

 29%|██▊       | 2/7 [03:39<09:05, 109.07s/it]

In [19]:
import os

In [20]:
save_path = '../../Data/brunch/sentence_pickles/dict_sentences_210621.pickle'

In [21]:
import pickle

In [22]:
with open(save_path, 'wb') as f:
    pickle.dump(txt_dict, f)

In [7]:
with open(save_path, 'rb') as f:
    txt_dict = pickle.load(f)

In [9]:
txt_dict['회식']

defaultdict(dict,
            {'spooky16-95': ['직장인에게 회식이란너를 보여줘지금은 코로나 덕분에 많이 사라진 직장 회식.회사 생활= 회식이라고 할 만큼 회식을 많이 하던 시절이 있었다.',
              '첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.',
              '자기들끼리 부어라 마셔라 하면서 한 마디씩 나에게 회사 생활이 어쩌고 저쩌고 했던 것 같은데 기억은 하나도 안 난다.',
              '(그만큼 쓸데없는 이야기가 많았던 듯.)나를 축하해 주려고 모인 게 아니라 그들은 그냥 술을 마실 핑계가 필요했던 것 같다.',
              '체질상 술을 잘 마시지도 못하는데 끊임없이 따라주는 술 때문에 힘들었다.',
              '그리고 결국 속이 안 좋아내 안의 것을 확인하고다음날 출근하면서 다시 그녀석을 확인할 때면 전날의 내가 얼마나 힘들었는지떠올랐다.',
              '그런데 시간이 지날수록 나는 은근히 회식을 즐기고 있었다.',
              '내가 술을 못 마시는 것 하나 빼고는 아저씨들의 일상을 관찰하는 게 꽤 재미있었다.',
              'A 대리님은 평소에 말이 별로 없는데 술만 들어가면 말을 많이 했다.',
              '은근히 재미있는 스타일인데 그분의 말발은 술이 들어가야 드러나곤 했다.',
              '물론 다음날은 여지없이 조용 모드. B 과장님은 술을 먹으면 귀여워지는 분이었다.',
              '아니 그렇게 귀여우신 분이 회사일 할 때는 점잖은 척하시다니. 술을 통해 본성을 찾아가는 게 신기했다.',
              'C 부서장님은 술자리에서 보이는 모습과 일상이 거의 똑같았다.',
              '그렇게 몸속에 알코올을 퍼붓는데도 다음날 생생해 보였다.',
     

In [83]:
html

'<html><head><style></style></head><body><h1>\'원격회식\' 들어보셨어요?</h1><br><h3>코로나19사태에 대처하는 스타트업 플랫팜의 자세</h3><br><br><h1 class="wrap_item item_type_text" data-app=\'{"type":"text","data":[{"type":"text","text":"전 직원 재택근무 중!"}],"size":"h1"}\'>전 직원 재택근무 중!</h1><br><br><span style="color:#333333;font-family:Georgia;background-color:#ffffff;">마스크 없이는 외출하기도 두려워지는 요즘. 코로나19의 여파와 마스크 대란 현상으로 최근 ‘</span>재택근무<span style="color:#333333;font-family:Georgia;background-color:#ffffff;">’를 실시하는 회사들도 많은데요. 이전부터 구성원들의 자율 원격근무 문화가 정착되어 있던 플랫팜 역시 <b>‘</b></span><b>전 직원 재택근무</b><span style="color:#333333;font-family:Georgia;background-color:#ffffff;"><b>’</b>로 코로나19에 대처하고 있습니다.</span><br><center><img src="http://t1.daumcdn.net/brunch/service/user/12kv/image/A3Bwu5iDVQEVG4raLZxUpGO0DD4.png"><br>화상 커뮤니케이션 서비스 ‘줌’을 통한 플랫팜의 근무 풍경(캡처)</center><br><br><h1 class="wrap_item item_type_text" data-app=\'{"type":"text","data":[{"type":"text","text":"화상 커뮤니케이션 서비스를 통한 자유로운 소통."}],"size":"h1"}\'>화상 커뮤니케이션 서비스를 통한 자유로운 소통.<

In [84]:
def escape_ansi(line):
    ansi_escape =re.compile(r'(\x9B|\x1B\[)[0-?]*[ -\/]*[@-~]')
    return ansi_escape.sub('', line)

In [85]:
escape_ansi(html)

'<html><head><style></style></head><body><h1>\'원격회식\' 들어보셨어요?</h1><br><h3>코로나19사태에 대처하는 스타트업 플랫팜의 자세</h3><br><br><h1 class="wrap_item item_type_text" data-app=\'{"type":"text","data":[{"type":"text","text":"전 직원 재택근무 중!"}],"size":"h1"}\'>전 직원 재택근무 중!</h1><br><br><span style="color:#333333;font-family:Georgia;background-color:#ffffff;">마스크 없이는 외출하기도 두려워지는 요즘. 코로나19의 여파와 마스크 대란 현상으로 최근 ‘</span>재택근무<span style="color:#333333;font-family:Georgia;background-color:#ffffff;">’를 실시하는 회사들도 많은데요. 이전부터 구성원들의 자율 원격근무 문화가 정착되어 있던 플랫팜 역시 <b>‘</b></span><b>전 직원 재택근무</b><span style="color:#333333;font-family:Georgia;background-color:#ffffff;"><b>’</b>로 코로나19에 대처하고 있습니다.</span><br><center><img src="http://t1.daumcdn.net/brunch/service/user/12kv/image/A3Bwu5iDVQEVG4raLZxUpGO0DD4.png"><br>화상 커뮤니케이션 서비스 ‘줌’을 통한 플랫팜의 근무 풍경(캡처)</center><br><br><h1 class="wrap_item item_type_text" data-app=\'{"type":"text","data":[{"type":"text","text":"화상 커뮤니케이션 서비스를 통한 자유로운 소통."}],"size":"h1"}\'>화상 커뮤니케이션 서비스를 통한 자유로운 소통.<

In [75]:
type(html)

str

In [74]:
html.encode('ascii',errors='ignore').strip('\n')

TypeError: a bytes-like object is required, not 'str'

* test

In [73]:
test=codecs.open('./scraped/휴가/7phs-11.html', 'r')
tmp = test.read()
print(tmp)

<html><head><style></style></head><body><h1>휴가의 의미</h1><br><h3>올해 첫 휴가를 사용한 후의 기록 </h3><br><h3>들어가며</h3><h4>며칠 전 올해 첫 '휴가'를 사용했습니다. 2021년이 시작되고 절반이 가까워지고 있는 시점에 첫 휴가를 사용했는데요, 오랜만에 휴가를 사용한 만큼 제겐 소중했고 여러 의미가 있었습니다. </h4><h4>저는 특별히 여행을 가거나, 큰 계획이 없어도 가끔씩 휴가를 사용하며 재충전 시간을 가지는 것을 선호하는 편이었어요. 작년까지는 한 달에 한 번은 휴가를 사용하는 것이 제게 큰 행복이었는데요, 올 해는 과거와는 다르게 휴가를 잘 사용하지 못하고 늦게 사용하게 되었습니다. </h4><h4>휴가를 사용 한 뒤 '휴가'에 대해서 제 생각을 정리해보았고, 여러분께 들려드릴까 해요. 읽으시면서 여러분의 휴가는 어떠신지 정리해보시고 저에게도 이야기 들려주세요. </h4><center><img src="http://t1.daumcdn.net/brunch/service/user/5sOI/image/XTrqL_UUwHKGmHEAbgn9xoiZ2dY.jpeg"><br></center><br><h3>1. 업무가 늘었어요(신규 업무 분장)  </h3><h4></h4><h4> 저는 기본적으로 '휴가'와 '일'은 적절하게 균형을 맞추면서 살아가는 것을 선호해요. "열심히 일한 당신, 떠나라!"라는 말도 좋아하는 편입니다. </h4><h4> 올해는 과거와는 조금 다른 균형을 맞추며 지내고 있어요. 먼저 작년보다 맡은 업무가 늘어나게 되었고, 업무 중에는 처음 해보게 되는 업무도 있었습니다. 기존에 제가 맡고 있던 MD 업무는 월말, 월초가 마감과 정산으로 바빴고 중순에는 휴가를 쓸 수 있었던 반면에 올해는 MD 업무와 함께 조직의 손익 및 경영관리 업무도 함께 되어 중순에도 자리를 비울 수 없게 되었습니다. 정리하면, 월말과 월초는 원래 바빴고 어쩌다 보니 올해는 중간에도 바쁘게 되었네요. </h

### cleaning (remove html tags)

In [5]:
def remove_tag(content):
   cleanr =re.compile('<.*?>')
   cleantext = re.sub(cleanr, '', content)
   return cleantext

In [6]:
save_path = './scraped/휴가'

In [7]:
for filename in os.listdir(save_path):
    if filename.endswith(".html"): 
        f = codecs.open(os.path.join(save_path, filename), 'r')
        raw_html = f.read()
        f.close()
        
        clean_html = remove_tag(raw_html)
        with open(os.path.join(save_path, filename)[:-4] + "txt", "w", encoding='UTF-8') as text_file:
            text_file.write(clean_html)
    else:
        continue

* test

In [75]:
print(remove_tag(tmp))

휴가의 의미올해 첫 휴가를 사용한 후의 기록 들어가며며칠 전 올해 첫 '휴가'를 사용했습니다. 2021년이 시작되고 절반이 가까워지고 있는 시점에 첫 휴가를 사용했는데요, 오랜만에 휴가를 사용한 만큼 제겐 소중했고 여러 의미가 있었습니다. 저는 특별히 여행을 가거나, 큰 계획이 없어도 가끔씩 휴가를 사용하며 재충전 시간을 가지는 것을 선호하는 편이었어요. 작년까지는 한 달에 한 번은 휴가를 사용하는 것이 제게 큰 행복이었는데요, 올 해는 과거와는 다르게 휴가를 잘 사용하지 못하고 늦게 사용하게 되었습니다. 휴가를 사용 한 뒤 '휴가'에 대해서 제 생각을 정리해보았고, 여러분께 들려드릴까 해요. 읽으시면서 여러분의 휴가는 어떠신지 정리해보시고 저에게도 이야기 들려주세요. 1. 업무가 늘었어요(신규 업무 분장)   저는 기본적으로 '휴가'와 '일'은 적절하게 균형을 맞추면서 살아가는 것을 선호해요. "열심히 일한 당신, 떠나라!"라는 말도 좋아하는 편입니다.  올해는 과거와는 조금 다른 균형을 맞추며 지내고 있어요. 먼저 작년보다 맡은 업무가 늘어나게 되었고, 업무 중에는 처음 해보게 되는 업무도 있었습니다. 기존에 제가 맡고 있던 MD 업무는 월말, 월초가 마감과 정산으로 바빴고 중순에는 휴가를 쓸 수 있었던 반면에 올해는 MD 업무와 함께 조직의 손익 및 경영관리 업무도 함께 되어 중순에도 자리를 비울 수 없게 되었습니다. 정리하면, 월말과 월초는 원래 바빴고 어쩌다 보니 올해는 중간에도 바쁘게 되었네요. 처음에는 새로운 업무를 진행하다 보니 휴가를 쓸 겨를도 없이 마무리하면 한 달이 지나가 있고, 분기도 지나가버렸습니다. 신규 업무의 경우에는 여러 동료들의 확인과 협의가 중요한 업무이기 때문에 동료들의 스케줄을 고려하게 되었고 휴가 사용은 점점 미뤄졌습니다. 또한 물어보고, 배워야 하는 부분들이 많아 쉽게 '휴가'를 사용하기 어려웠습니다.  2. 하다 보니 재미있더라     2021년 1분기는 신규업무에 정신없이 적응하니 끝나 있었고, 2분기가 되니 업무가 